# 处理特征结构

NLTK 中的特征结构使用构造函数 [nltk.FeatStruct](http://www.nltk.org/_modules/nltk/featstruct.html#FeatStruct) 声明，原子特征值可以是字符串或整数：

In [1]:
import nltk

fs1 = nltk.FeatStruct(TENSE='past', NUM='sg')
print(fs1)

[ NUM   = 'sg'   ]
[ TENSE = 'past' ]


一个特征结构实际上只是一种字典，所以我们可以通过索引访问它的值，也可以指定某些特征的值：

In [2]:
fs1 = nltk.FeatStruct(PER=3, NUM='pl', GND='fem')
print(fs1['GND'])
fs1['CASE'] = 'acc'

fem


我们还可以为特征结构定义更复杂的值：

In [3]:
fs2 = nltk.FeatStruct(POS='N', AGR=fs1)
print(fs2, end='\n\n')
print(fs2['AGR'], end='\n\n')
print(fs2['AGR']['PER'])

[       [ CASE = 'acc' ] ]
[ AGR = [ GND  = 'fem' ] ]
[       [ NUM  = 'pl'  ] ]
[       [ PER  = 3     ] ]
[                        ]
[ POS = 'N'              ]

[ CASE = 'acc' ]
[ GND  = 'fem' ]
[ NUM  = 'pl'  ]
[ PER  = 3     ]

3


指定特征结构的另一种方法是使用方括号括起来包含 feature = value 格式的特征-值对的的字符串，其中值本身可能是特征结构：

In [4]:
print(nltk.FeatStruct("[POS='N', AGR=[PER=3, NUM='pl', GND='fem']]"))

[       [ GND = 'fem' ] ]
[ AGR = [ NUM = 'pl'  ] ]
[       [ PER = 3     ] ]
[                       ]
[ POS = 'N'             ]


特征结构本身不依赖于语言对象，它们是表示知识的通用结构。例如：我们可以将要给人的信息用特征结构编码：

In [5]:
print(nltk.FeatStruct(NAME='Lee', TELNO='01 27 86 42 96', AGE=33))

[ AGE   = 33               ]
[ NAME  = 'Lee'            ]
[ TELNO = '01 27 86 42 96' ]


我们可以将特征结构作为**有向无环图**（directed acyclic graphs，DAGs）来查看，特征名称作为弧上的标签出现，特征值作为弧指向的节点的标签出现：

![dag01.png](resources/dag01.png)

特征值也可以是复杂类型的，如下图。我们用标签的元组来表示路径，那么（'ADDRESS', 'STREET'）就是一个特征路径，它的值是标签为 rue Pascal 的节点。

![dag02.png](resources/dag02.png)

现在我们考虑这么一种情况：Lee 有一个配偶叫做 Kim，Kim 的地址和 Lee 相同，可以如下图这样表示：

![dag03.png](resources/dag03.png)

我们可以简化上图，让不同的弧“共享”同一个子图：

![dag04.png](resources/dag04.png)

这种共享子图的方式被称为**结构共享**或**重入**，在 NLTK 中我们可以在共享的特征结构第一次出现的地方加上一个括号阔起的数字前缀，例如 (1) 来实现它，以后任何对这个结构的引用将使用符号 -> (1)：

In [6]:
print(nltk.FeatStruct("""[NAME='Lee', ADDRESS=(1)[NUM=74, STREET='rue Pascal'],
                        SPOUSE=[NAME='Kim', ADDRESS->(1)]]"""))

[ ADDRESS = (1) [ NUM    = 74           ] ]
[               [ STREET = 'rue Pascal' ] ]
[                                         ]
[ NAME    = 'Lee'                         ]
[                                         ]
[ SPOUSE  = [ ADDRESS -> (1)  ]           ]
[           [ NAME    = 'Kim' ]           ]


## 包含和统一

特征结构可以按照其通用的程度进行排序，例如在下面的例子中，a 比 b 更一般（更少特征），b 比 c 更一般。这个顺序被称为**包含**，更一般的说法是，如果 FS1 包含了 FS0 中的所有信息，那么我们就说 FS0 被 FS1 包含，用符号 ⊑ 来表示：

    a. [NUMBER = 74]

    b. [NUMBER = 74          ]
       [STREET = 'rue Pascal']

    c. [NUMBER = 74          ]
       [STREET = 'rue Pascal']
       [CITY = 'Paris'       ]
       
如果 FSO 被 FS1 所包含，即 FS0 ⊑ FS1，那么 FS1 必须具备 FS0 的所有路径，同时它也可能有其自己的额外路径。显而易见，d 既不包含 a 也不被 a 包含：
       
    d. [TELNO = 01 27 86 42 96]
  
综上所述，某些特征结构会比其它的包含更多的信息，那么我们如何将这么多出来的信息添加给指定的特征结构呢？例如：我们觉得决定地址应该包含的不只是门牌号和街道名称，还应该包含城市名，也就是说我们要把下面例子中的图 b 合并到图 a 上产生图 c：
    
a. ![dag04-1.png](resources/dag04-1.png)

b. ![dag04-2.png](resources/dag04-2.png)

c. ![dag04-3.png](resources/dag04-3.png)

合并两个特征结构的信息被称为**统一**，由方法 unify() 来实现：

In [7]:
fs1 = nltk.FeatStruct(NUMBER=74, STREET='rue Pascal')
fs2 = nltk.FeatStruct(CITY='Pairs')
print(fs1.unify(fs2))

[ CITY   = 'Pairs'      ]
[ NUMBER = 74           ]
[ STREET = 'rue Pascal' ]


统一的符号化表示为 FS0 ⊔ FS1，它是一个对称操作，所以 FS0 ⊔ FS1 = FS1 ⊔ FS0：

In [8]:
print(fs2.unify(fs1))

[ CITY   = 'Pairs'      ]
[ NUMBER = 74           ]
[ STREET = 'rue Pascal' ]


如果统一两个特征结构具有包含关系，那么统一的结果是更具体的那个特征结构，也就是说如果 FS0 ⊑ FS1，那么 FS0 ⊔ FS1 = FS1；如果统一的两个特征结构共享某个路径，但路径的值不同，那么统一将会失败，并返回结果 None：

In [9]:
fs0 = nltk.FeatStruct(A='a')
fs1 = nltk.FeatStruct(A='b')
print(fs0.unify(fs1))

None


现在，我们来看一下统一如何与结构共享相互作用的：

In [10]:
fs0 = nltk.FeatStruct("""[NAME=Lee,
                          ADDRESS=[NUMBER=74,
                                   STREET='rue Pascal'],
                          SPOUSE=[NAME=Kim,
                                  ADDRESS=[NUMBER=74,
                                           STREET='rue Pascal']]]""")
print(fs0)

[ ADDRESS = [ NUMBER = 74           ]               ]
[           [ STREET = 'rue Pascal' ]               ]
[                                                   ]
[ NAME    = 'Lee'                                   ]
[                                                   ]
[           [ ADDRESS = [ NUMBER = 74           ] ] ]
[ SPOUSE  = [           [ STREET = 'rue Pascal' ] ] ]
[           [                                     ] ]
[           [ NAME    = 'Kim'                     ] ]


接下来我们为 Kim 的地址指定一个 CITY 作为参数，请注意 fs1 需要包含从特征结构的根节点到 CITY 的整个路径：

In [11]:
fs1 = nltk.FeatStruct("[SPOUSE = [ADDRESS = [CITY = Paris]]]")
print(fs1.unify(fs0))

[ ADDRESS = [ NUMBER = 74           ]               ]
[           [ STREET = 'rue Pascal' ]               ]
[                                                   ]
[ NAME    = 'Lee'                                   ]
[                                                   ]
[           [           [ CITY   = 'Paris'      ] ] ]
[           [ ADDRESS = [ NUMBER = 74           ] ] ]
[ SPOUSE  = [           [ STREET = 'rue Pascal' ] ] ]
[           [                                     ] ]
[           [ NAME    = 'Kim'                     ] ]


我们把 fs0 替换为 fs2 这一结构共享版本，可以看出两个地方的 ADDRESS 都添加了 CITY 特征：

In [12]:
fs2 = nltk.FeatStruct("""[NAME=Lee, ADDRESS=(1)[NUMBER=74, STREET='rue Pascal'],
                          SPOUSE=[NAME=Kim, ADDRESS->(1)]]""")
print(fs1.unify(fs2))

[               [ CITY   = 'Paris'      ] ]
[ ADDRESS = (1) [ NUMBER = 74           ] ]
[               [ STREET = 'rue Pascal' ] ]
[                                         ]
[ NAME    = 'Lee'                         ]
[                                         ]
[ SPOUSE  = [ ADDRESS -> (1)  ]           ]
[           [ NAME    = 'Kim' ]           ]


结构共享也可以使用变量表示，如 ?n：

In [13]:
fs1 = nltk.FeatStruct("[ADDRESS1=[NUMBER=74, STREET='rue Pascal']]")
fs2 = nltk.FeatStruct("[ADDRESS1=?n, ADDRESS2=?n]")
print(fs2, end='\n\n')
print(fs2.unify(fs1))

[ ADDRESS1 = ?n ]
[ ADDRESS2 = ?n ]

[ ADDRESS1 = (1) [ NUMBER = 74           ] ]
[                [ STREET = 'rue Pascal' ] ]
[                                          ]
[ ADDRESS2 -> (1)                          ]
